In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import math
import statistics
from math import nan, isnan
import glob
import os



path = '/content/drive/MyDrive/RCData/'

all_files = glob.glob(path+'*.csv')

for filename in all_files:



  #Reading data
  Origin = pd.read_csv(filename,usecols=['ID','Task_Order','JS_Task','Buttons1','GMBM_Set','Positive_Order',
                                                'Person_Task_Order','Buttons2','SEX','Education','Age',
                                                        '1Shape','1Congruence','1RT','1Score',
                                                        '2Character','2Shape','2Congruence','2RT',
                                                        '2Score','Health group','Treatment','BDI_Total','GAD7_Total','RRS_Total','Rosenberg_Total2',
                                                  'Cognitive_Total','Somatic_af_Total','G_me_RT_Matched','B_me_RT_Matched','G_other_RT_Matched','B_other_RT_Matched',
                                                  'Self_RT_Matched','Other_RT_Matched','TimeDur_POSself','TimeDur_NEGself','TimeDur_POSother','TimeDur_NEGother'])




  #Function to get the mean of a list
  def Average(lst):
    return sum(lst) / len(lst)


  #Created a function to calculate variance, the one in the statistics module seemed to not work with my data for an unknown reason
  def variance(data):
    # Number of observations
    n = len(data)
    # Mean of the data
    mean = sum(data) / n
    # Square deviations
    deviations = [(x - mean) ** 2 for x in data]
    # Variance
    variance = sum(deviations) / n
    return variance




  #EZ Diffusion model code taken from the internet, tested and works the same as Wagenmakers online tool
  # EZ Diffusion model functions
  def logit(p):
      return math.log(p/(1-p))

  def ezdiff(MRT,VRT,p):

      if p == 0:
          print("Oops, only errors for subject "+ "!")
      elif p == 0.5:
          print("Oops, chance performance for "+ "!")
      elif p == 1:
          print("Oops,  only correct responses for "+ "!")
      
      s = 0.1
      s2 = s*s # scaling parameter squared
      L = logit(p)
      x1 = (L*p*p - L*p + p - 0.5)
      x = L * x1 / VRT
      v = np.sign(p - 0.5) * s*math.pow(x,0.25)
      a = s2*logit(p)/v
      y = -v*a/s2
      MDT = (a/(2*v))*(1-math.exp(y))/(1+math.exp(y))
      Ter = MRT - MDT
      
      return([v,a,Ter])


  #Time duration sum code, this calculates the amount of time taken for participants to write word attributions for GMBMGOBO

  GM_Time = Origin['TimeDur_POSself'].sum()
  BM_Time = Origin['TimeDur_NEGself'].sum()
  GO_Time = Origin['TimeDur_POSother'].sum()
  BO_Time = Origin['TimeDur_NEGother'].sum()


  #Percentage correct
  Self_Percent = Origin.groupby(["1Shape","1Congruence"])["1Score"].mean()[0]
  Other_Percent = Origin.groupby(["1Shape","1Congruence"])["1Score"].mean()[3]

  BM_Percent = Origin.groupby(["2Shape","2Congruence"])["2Score"].mean()[0]
  GM_Percent = Origin.groupby(["2Shape","2Congruence"])["2Score"].mean()[4]
  BO_Percent = Origin.groupby(["2Shape","2Congruence"])["2Score"].mean()[2]
  GO_Percent = Origin.groupby(["2Shape","2Congruence"])["2Score"].mean()[6]


  #These if statements add artificial error into perfectly correct scores by a degree of 1% so that the EZ algorithm works
  if Self_Percent == 1:
    Self_Percent = Self_Percent - 0.01

  if Self_Percent == 0.5:
    Self_Percent = Self_Percent - 0.01

  if Other_Percent == 1:
    Other_Percent = Other_Percent - 0.01

  if Other_Percent == 0.5:
    Other_Percent = Other_Percent - 0.01

  if GM_Percent == 1:
    GM_Percent = GM_Percent - 0.01

  if GM_Percent == 0.5:
    GM_Percent = GM_Percent - 0.01

  if BM_Percent == 1:
    BM_Percent = BM_Percent - 0.01

  if BM_Percent == 0.5:
    BM_Percent = BM_Percent - 0.01

  if GO_Percent == 1:
    GO_Percent = GO_Percent - 0.01

  if GO_Percent == 0.5:
    GO_Percent = GO_Percent - 0.01

  if BO_Percent == 1:
    BO_Percent = BO_Percent - 0.01

  if BO_Percent == 0.5:
    BO_Percent = BO_Percent - 0.01




  #Mean reaction time
  Self_RT = Origin['Self_RT_Matched'][0]
  Other_RT = Origin['Other_RT_Matched'][0]
  B_me_RT = Origin['B_me_RT_Matched'][0]
  G_other_RT = Origin['G_other_RT_Matched'][0]
  G_me_RT = Origin['G_me_RT_Matched'][0]
  B_other_RT = Origin['B_other_RT_Matched'][0]


  #Variance list preparation

  Self_RT_Lists = Origin.loc[(Origin['1Shape'] == 'Self') & (Origin['1Congruence'] == 'congruent')]
  SRL = Self_RT_Lists['1RT']
  SRL_List = SRL.values.tolist()
  SRT_list = SRL_List
  Self_RT_Numbers = [item for item in SRT_list if not(pd.isnull(item)) == True]

  Stranger_RT_Lists = Origin.loc[(Origin['1Shape'] == 'Stranger') & (Origin['1Congruence'] == 'congruent')]
  ORL = Stranger_RT_Lists['1RT']
  ORL_List = ORL.values.tolist()
  ORT_list = ORL_List
  Stranger_RT_Numbers = [item for item in ORT_list if not(pd.isnull(item)) == True]

  GM_RT_Lists = Origin.loc[(Origin['2Shape'] == 'Good_me') & (Origin['2Congruence'] == 'congruent')]
  GMRL = GM_RT_Lists['2RT']
  GMRL_List = GMRL.values.tolist()
  GMRT_list = GMRL_List
  GM_RT_Numbers = [item for item in GMRT_list if not(pd.isnull(item)) == True]

  BM_RT_Lists = Origin.loc[(Origin['2Shape'] == 'Bad_me') & (Origin['2Congruence'] == 'congruent')]
  BMRL = BM_RT_Lists['2RT']
  BMRL_List = BMRL.values.tolist()
  BMRT_list = BMRL_List
  BM_RT_Numbers = [item for item in BMRT_list if not(pd.isnull(item)) == True]


  GO_RT_Lists = Origin.loc[(Origin['2Shape'] == 'Good_other') & (Origin['2Congruence'] == 'congruent')]
  GORL = GO_RT_Lists['2RT']
  GORL_List = GORL.values.tolist()
  GORT_list = GORL_List
  GO_RT_Numbers = [item for item in GORT_list if not(pd.isnull(item)) == True]


  BO_RT_Lists = Origin.loc[(Origin['2Shape'] == 'Bad_other') & (Origin['2Congruence'] == 'congruent')]
  BORL = BO_RT_Lists['2RT']
  BORL_List = BORL.values.tolist()
  BORT_list = BORL_List
  BO_RT_Numbers = [item for item in BORT_list if not(pd.isnull(item)) == True]

  #Variance

  Self_Variance = variance(Self_RT_Numbers)
  Stranger_Variance = variance(Stranger_RT_Numbers)
  GM_Variance = variance(GM_RT_Numbers)
  BM_Variance = variance(BM_RT_Numbers)
  GO_Variance = variance(GO_RT_Numbers)
  BO_Variance = variance(BO_RT_Numbers)


  #EZ-diffusion calculation

  Self_v = ezdiff(Self_RT,Self_Variance,Self_Percent)[0]
  Stranger_v = ezdiff(Other_RT,Stranger_Variance,Other_Percent)[0]

  v_bias = Stranger_v - Self_v

  GM_v = ezdiff(G_me_RT,GM_Variance,GM_Percent)[0]
  BM_v = ezdiff(B_me_RT,BM_Variance,BM_Percent)[0]

  GO_v = ezdiff(G_other_RT,GO_Variance,GO_Percent)[0]
  BO_v = ezdiff(B_other_RT,BO_Variance,BO_Percent)[0]

  Self_a = ezdiff(Self_RT,Self_Variance,Self_Percent)[1]
  Stranger_a = ezdiff(Other_RT,Stranger_Variance,Other_Percent)[1]

  Self_Ter = ezdiff(Self_RT,Self_Variance,Self_Percent)[2]
  Stranger_Ter = ezdiff(Other_RT,Stranger_Variance,Other_Percent)[2]

  GM_a = ezdiff(G_me_RT,GM_Variance,GM_Percent)[1]
  BM_a = ezdiff(B_me_RT,BM_Variance,BM_Percent)[1]

  GO_a = ezdiff(G_other_RT,GO_Variance,GO_Percent)[1]
  BO_a = ezdiff(B_other_RT,BO_Variance,BO_Percent)[1]

  GM_Ter = ezdiff(G_me_RT,GM_Variance,GM_Percent)[2]
  BM_Ter = ezdiff(B_me_RT,BM_Variance,BM_Percent)[2]

  GO_Ter = ezdiff(G_other_RT,GO_Variance,GO_Percent)[2]
  BO_Ter = ezdiff(B_other_RT,BO_Variance,BO_Percent)[2]


  #Valence and person variable creator

  Valence_good = ((G_me_RT + G_other_RT)/2)
  Valence_bad = ((B_me_RT + B_other_RT)/2)
  Person_self = ((G_me_RT + B_me_RT)/2)
  Person_other = ((G_other_RT + B_other_RT)/2)

  #Cleaning data by removing RT's +/- 3 std. away from mean

  Self_Mean = Origin.groupby(["1Shape","1Congruence"])["1RT"].mean()[0]
  Other_Mean = Origin.groupby(["1Shape","1Congruence"])["1RT"].mean()[3]

  BM_Mean = Origin.groupby(["2Shape","2Congruence"])["2RT"].mean()[0]
  GM_Mean = Origin.groupby(["2Shape","2Congruence"])["2RT"].mean()[4]
  BO_Mean = Origin.groupby(["2Shape","2Congruence"])["2RT"].mean()[2]
  GO_Mean = Origin.groupby(["2Shape","2Congruence"])["2RT"].mean()[6]



  Self_STD = Origin.groupby(["1Shape","1Congruence"])["1RT"].std()[0]
  Other_STD = Origin.groupby(["1Shape","1Congruence"])["1RT"].std()[3]

  BM_STD = Origin.groupby(["2Shape","2Congruence"])["2RT"].std()[0]
  GM_STD = Origin.groupby(["2Shape","2Congruence"])["2RT"].std()[4]
  BO_STD = Origin.groupby(["2Shape","2Congruence"])["2RT"].std()[2]
  GO_STD = Origin.groupby(["2Shape","2Congruence"])["2RT"].std()[6]


  Self_upper_limit = Self_Mean + 3*Self_STD
  Self_lower_limit = Self_Mean + -3*Self_STD

  Other_upper_limit = Other_Mean + 3*Other_STD
  Other_lower_limit = Other_Mean + -3*Other_STD

  BM_upper_limit = BM_Mean + 3*BM_STD
  BM_lower_limit = BM_Mean + -3*BM_STD

  GM_upper_limit = GM_Mean + 3*GM_STD
  GM_lower_limit = GM_Mean + -3*GM_STD

  BO_upper_limit = BO_Mean + 3*BO_STD
  BO_lower_limit = BO_Mean + -3*BO_STD

  GO_upper_limit = GO_Mean + 3*GO_STD
  GO_lower_limit = GO_Mean + -3*GO_STD

  #Cleaned lists

  toofast = 0.2

  Self_clean = ([i for i in Self_RT_Numbers if i > Self_lower_limit and i < Self_upper_limit and i > toofast])
  Other_clean = ([i for i in Stranger_RT_Numbers if i > Other_lower_limit and i < Other_upper_limit and i > toofast])

  BM_clean = ([i for i in BM_RT_Numbers if i > BM_lower_limit and i < BM_upper_limit and i > toofast])
  GM_clean = ([i for i in GM_RT_Numbers if i > GM_lower_limit and i < GM_upper_limit and i > toofast])

  BO_clean = ([i for i in BO_RT_Numbers if i > BO_lower_limit and i < BO_upper_limit and i > toofast])
  GO_clean = ([i for i in GO_RT_Numbers if i > GO_lower_limit and i < GO_upper_limit and i > toofast])

  #Cleaned Mean variables
  Self_clean_Mean = Average(Self_clean)
  Other_clean_Mean = Average(Other_clean)

  BM_clean_Mean = Average(BM_clean)
  GM_clean_Mean = Average(GM_clean)
  BO_clean_Mean = Average(BO_clean)
  GO_clean_Mean = Average(GO_clean)

  #Cleaned variance

  bSelf_Variance = variance(Self_clean)
  bStranger_Variance = variance(Other_clean)
  bGM_Variance = variance(GM_clean)
  bBM_Variance = variance(BM_clean)
  bGO_Variance = variance(GO_clean)
  bBO_Variance = variance(BO_clean)

  #Cleaned EZ-Diffusion modelling

  Self_v2 = ezdiff(Self_clean_Mean,bSelf_Variance,Self_Percent)[0]
  Stranger_v2 = ezdiff(Other_clean_Mean,bStranger_Variance,Other_Percent)[0]

  v_bias2 = Stranger_v2 - Self_v2

  GM_v2 = ezdiff(GM_clean_Mean,bGM_Variance,GM_Percent)[0]
  BM_v2 = ezdiff(BM_clean_Mean,bBM_Variance,BM_Percent)[0]

  GO_v2 = ezdiff(GO_clean_Mean,bGO_Variance,GO_Percent)[0]
  BO_v2 = ezdiff(BO_clean_Mean,bBO_Variance,BO_Percent)[0]

  Self_a2 = ezdiff(Self_clean_Mean,bSelf_Variance,Self_Percent)[1]
  Stranger_a2 = ezdiff(Other_clean_Mean,bStranger_Variance,Other_Percent)[1]

  Self_Ter2 = ezdiff(Self_clean_Mean,bSelf_Variance,Self_Percent)[2]
  Stranger_Ter2 = ezdiff(Other_clean_Mean,bStranger_Variance,Other_Percent)[2]

  GM_a2 = ezdiff(GM_clean_Mean,bGM_Variance,GM_Percent)[1]
  BM_a2 = ezdiff(BM_clean_Mean,bBM_Variance,BM_Percent)[1]

  GO_a2 = ezdiff(GO_clean_Mean,bGO_Variance,GO_Percent)[1]
  BO_a2 = ezdiff(BO_clean_Mean,bBO_Variance,BO_Percent)[1]

  GM_Ter2 = ezdiff(GM_clean_Mean,bGM_Variance,GM_Percent)[2]
  BM_Ter2 = ezdiff(BM_clean_Mean,bBM_Variance,BM_Percent)[2]

  GO_Ter2 = ezdiff(GO_clean_Mean,bGO_Variance,GO_Percent)[2]
  BO_Ter2 = ezdiff(BO_clean_Mean,bBO_Variance,BO_Percent)[2]

  #Inserting the drift values into the excel file

  Origin.insert(0, "Self_v",Self_v, True)
  Origin.insert(1, "Stranger_v",Stranger_v, True)
  Origin.insert(2, "v_bias",v_bias, True)
  Origin.insert(3, "GM_v",GM_v, True)
  Origin.insert(4, "BM_v",BM_v, True)
  Origin.insert(5, "GO_v",GO_v, True)
  Origin.insert(6, "BO_v",BO_v, True)

  Origin.insert(7, "Self_a",Self_a, True)
  Origin.insert(8, "Stranger_a",Stranger_a, True)

  Origin.insert(9, "GM_a",GM_a, True)
  Origin.insert(10, "BM_a",BM_a, True)
  Origin.insert(11, "GO_a",GO_a, True)
  Origin.insert(12, "BO_a",BO_a, True)

  Origin.insert(13, "Self_Ter",Self_v, True)
  Origin.insert(14, "Stranger_Ter",Stranger_v, True)

  Origin.insert(15, "GM_Ter",GM_Ter, True)
  Origin.insert(16, "BM_Ter",BM_Ter, True)
  Origin.insert(17, "GO_Ter",GO_Ter, True)
  Origin.insert(18, "BO_Ter",BO_Ter, True)

  #Inserting time sum codes

  Origin.insert(19, "GM_Time",GM_Time, True)
  Origin.insert(20, "BM_Time",BM_Time, True)
  Origin.insert(21, "GO_Time",GO_Time, True)
  Origin.insert(22, "BO_Time",BO_Time, True)

  #Inserting percent correct

  Origin.insert(23, "GM_Percent",GM_Percent, True)
  Origin.insert(24, "BM_Percent",BM_Percent, True)
  Origin.insert(25, "GO_Percent",GO_Percent, True)
  Origin.insert(26, "BO_Percent",BO_Percent, True)

  Origin.insert(27, "Self_Percent",Self_Percent, True)
  Origin.insert(28, "Other_Percent",Other_Percent, True)

  #Inserting valence and person variables
 
  Origin.insert(29, "Valence_good",Valence_good, True)
  Origin.insert(30, "Valence_bad",Valence_bad, True)
  Origin.insert(31, "Person_self",Person_self, True)
  Origin.insert(32, "Person_other",Person_other, True)

  #Inserting std. cleaned mean values


  Origin.insert(33, "Self_clean_Mean",Self_clean_Mean, True)
  Origin.insert(34, "Other_clean_Mean",Other_clean_Mean, True)

  Origin.insert(35, "GM_clean_Mean",GM_clean_Mean, True)
  Origin.insert(36, "BM_clean_Mean",BM_clean_Mean, True)
  Origin.insert(37, "GO_clean_Mean",GO_clean_Mean, True)
  Origin.insert(38, "BO_clean_Mean",BO_clean_Mean, True)
  
  #Inserting the cleaned EZ model data into the data frame

  Origin.insert(39, "Self_v2",Self_v2, True)
  Origin.insert(40, "Stranger_v2",Stranger_v2, True)
  Origin.insert(41, "v_bias2",v_bias2, True)

  Origin.insert(42, "GM_v2",GM_v2, True)
  Origin.insert(43, "BM_v2",BM_v2, True)
  Origin.insert(44, "GO_v2",GO_v2, True)
  Origin.insert(45, "BO_v2",BO_v2, True)

  Origin.insert(46, "Self_a2",Self_a2, True)
  Origin.insert(8, "Stranger_a2",Stranger_a2, True)

  Origin.insert(47, "GM_a2",GM_a2, True)
  Origin.insert(48, "BM_a2",BM_a2, True)
  Origin.insert(49, "GO_a2",GO_a2, True)
  Origin.insert(50, "BO_a2",BO_a2, True)

  Origin.insert(51, "Self_Ter2",Self_Ter2, True)
  Origin.insert(52, "Stranger_Ter2",Stranger_Ter2, True)

  Origin.insert(53, "GM_Ter2",GM_Ter2, True)
  Origin.insert(54, "BM_Ter2",BM_Ter2, True)
  Origin.insert(55, "GO_Ter2",GO_Ter2, True)
  Origin.insert(56, "BO_Ter2",BO_Ter2, True)

  #Shift excess iterations up (831 results for some reason)
  
  Origin["Self_v2"] = Origin["Self_v2"].shift(-829)
  Origin["Stranger_v2"] = Origin["Stranger_v2"].shift(-829)
  Origin["v_bias2"] = Origin["v_bias2"].shift(-829)

  Origin["GM_v2"] = Origin["GM_v2"].shift(-829)
  Origin["GO_v2"] = Origin["GO_v2"].shift(-829)
  Origin["BO_v2"] = Origin["BO_v2"].shift(-829)
  Origin["BM_v2"] = Origin["BM_v2"].shift(-829)

  Origin["Self_a2"] = Origin["Self_a2"].shift(-829)
  Origin["Stranger_a2"] = Origin["Stranger_a2"].shift(-829)

  Origin["GM_a2"] = Origin["GM_a2"].shift(-829)
  Origin["GO_a2"] = Origin["GO_a2"].shift(-829)
  Origin["BO_a2"] = Origin["BO_a2"].shift(-829)
  Origin["BM_a2"] = Origin["BM_a2"].shift(-829)

  Origin["Self_Ter2"] = Origin["Self_Ter2"].shift(-829)
  Origin["Stranger_Ter2"] = Origin["Stranger_Ter2"].shift(-829)

  Origin["GM_Ter2"] = Origin["GM_Ter2"].shift(-829)
  Origin["GO_Ter2"] = Origin["GO_Ter2"].shift(-829)
  Origin["BO_Ter2"] = Origin["BO_Ter2"].shift(-829)
  Origin["BM_Ter2"] = Origin["BM_Ter2"].shift(-829)

  
  Origin["Self_clean_Mean"] = Origin["Self_clean_Mean"].shift(-829)
  Origin["Other_clean_Mean"] = Origin["Other_clean_Mean"].shift(-829)

  Origin["GM_clean_Mean"] = Origin["GM_clean_Mean"].shift(-829)
  Origin["BM_clean_Mean"] = Origin["BM_clean_Mean"].shift(-829)
  Origin["BO_clean_Mean"] = Origin["BO_clean_Mean"].shift(-829)
  Origin["GO_clean_Mean"] = Origin["GO_clean_Mean"].shift(-829)


  
  Origin["Valence_good"] = Origin["Valence_good"].shift(-829)
  Origin["Valence_bad"] = Origin["Valence_bad"].shift(-829)
  Origin["Person_self"] = Origin["Person_self"].shift(-829)
  Origin["Person_other"] = Origin["Person_other"].shift(-829)



  Origin["Self_Percent"] = Origin["Self_Percent"].shift(-829)
  Origin["Other_Percent"] = Origin["Other_Percent"].shift(-829)

  Origin["GM_Percent"] = Origin["GM_Percent"].shift(-829)
  Origin["BM_Percent"] = Origin["BM_Percent"].shift(-829)
  Origin["GO_Percent"] = Origin["GO_Percent"].shift(-829)
  Origin["BO_Percent"] = Origin["BO_Percent"].shift(-829)


  Origin["Self_v"] = Origin["Self_v"].shift(-829)
  Origin["Stranger_v"] = Origin["Stranger_v"].shift(-829)
  Origin["v_bias"] = Origin["v_bias"].shift(-829)
  Origin["GM_v"] = Origin["GM_v"].shift(-829)
  Origin["GO_v"] = Origin["GO_v"].shift(-829)
  Origin["BO_v"] = Origin["BO_v"].shift(-829)
  Origin["BM_v"] = Origin["BM_v"].shift(-829)

  Origin["Self_a"] = Origin["Self_a"].shift(-829)
  Origin["Stranger_a"] = Origin["Stranger_a"].shift(-829)

  Origin["GM_a"] = Origin["GM_a"].shift(-829)
  Origin["GO_a"] = Origin["GO_a"].shift(-829)
  Origin["BO_a"] = Origin["BO_a"].shift(-829)
  Origin["BM_a"] = Origin["BM_a"].shift(-829)

  Origin["Self_Ter"] = Origin["Self_Ter"].shift(-829)
  Origin["Stranger_Ter"] = Origin["Stranger_Ter"].shift(-829)

  Origin["GM_Ter"] = Origin["GM_Ter"].shift(-829)
  Origin["GO_Ter"] = Origin["GO_Ter"].shift(-829)
  Origin["BO_Ter"] = Origin["BO_Ter"].shift(-829)
  Origin["BM_Ter"] = Origin["BM_Ter"].shift(-829)

  Origin["GM_Time"] = Origin["GM_Time"].shift(-829)
  Origin["GO_Time"] = Origin["GO_Time"].shift(-829)
  Origin["BO_Time"] = Origin["BO_Time"].shift(-829)
  Origin["BM_Time"] = Origin["BM_Time"].shift(-829)

  #Deleting unwanted columns
  unwanted_columns = ['1Shape','1Congruence','1RT','1Score',
                                                        '2Character','2Shape','2Congruence','2RT',
                                                        '2Score','TimeDur_POSself','TimeDur_NEGself','TimeDur_POSother','TimeDur_NEGother']

  Origin.drop(unwanted_columns, inplace=True, axis=1)



  #This creates a file name that is relevant to the data
  FileName = (str('EZRC_4') + str(Origin['ID'][0]) + str(Origin['SEX'][0]) + str(Origin["Age"][0]) + str(Origin['Health group'][0]) +str('.csv'))
  #Save file
  Origin.to_csv(r'/content/drive/MyDrive/EZ5/%s'%FileName, index = False)

print('progress...')



progress...


In [ ]:
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join("/content/drive/MyDrive/EZ_Outliers/", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=False)
df.dropna()

df.to_excel(r'/content/drive/MyDrive/CData/Cleaned_Results.xlsx', index = False)

Resultant CSV after joining all CSV files at a particular location...
